# Import data and module

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [2]:
trainv1 = pd.read_csv("train_cleaned_version1.csv", low_memory=False, index_col=0)
testv1 = pd.read_csv("test_cleaned_1.csv", low_memory=False, index_col=0)

# Standardization (略)
由於表現較好的XGBOOST、random forest為tree-based model；另外，Logistic regression只要不做regularization像是Lasso、Ridge，理論基礎上有沒有standardization無差別。而KNN是這裡唯一會受重大影響的，但上傳次數有限就先不理它。(這個資料集以名目順序為主，應該也不太適合)

注意:其實如果要Standardization的話要對train test split後的train資料做，而不是一起做

# One-hot encoding
1. Tree-based model不適合用one-hot encoding。BJ4詳見:https://towardsdatascience.com/one-hot-encoding-is-making-your-tree-based-ensembles-worse-heres-why-d64b282b5769

2. Logistic regression、KNN則需要

Note:僅針對名目尺度處理(如果只有2類(ex.GENDER)已經處理過了，這裡只要把多類轉換就行)
原來就是名目:收費地址CHARGE_CITY_CD、聯絡地址CONTACT_CITY_CD、婚姻狀況MARRIAGE_CD 、九大客群CUST_9_SEGMENTS_CD
補遺失值後視為名目:首次擔任要保人年齡(級距)APC_1ST_AGE、再購次數(級距)REBUY_TIMES_CNT、曾投保主約件數(等級)RFM_M_LEVEL、APC_1ST_YEARDIF、TERMINATION_RATE、RFM_R、LEVEL

Note:職業類別特殊處理=>將0->7視為順序尺度

注意:One hot 是可以在train_test split前做的

In [3]:
def occ_tran(col):
    if col == 0:
        return 7
    else:
        return col

In [4]:
trainv1["OCCUPATION_CLASS_CD"] = trainv1["OCCUPATION_CLASS_CD"].apply(occ_tran)

In [5]:
# trainv1.to_csv("train_cleaned_version1_proc_tree.csv")

In [6]:
A = pd.get_dummies(trainv1["CHARGE_CITY_CD"], drop_first=True)
B = pd.get_dummies(trainv1["CONTACT_CITY_CD"], drop_first=True)
C = pd.get_dummies(trainv1["MARRIAGE_CD"], drop_first=True)
D = pd.get_dummies(trainv1["CUST_9_SEGMENTS_CD"], drop_first=True)
E = pd.get_dummies(trainv1["APC_1ST_AGE"], drop_first=True)
F = pd.get_dummies(trainv1["REBUY_TIMES_CNT"], drop_first=True)
G = pd.get_dummies(trainv1["RFM_M_LEVEL"], drop_first=True)
H = pd.get_dummies(trainv1["APC_1ST_YEARDIF"], drop_first=True)
I = pd.get_dummies(trainv1["TERMINATION_RATE"], drop_first=True)
J = pd.get_dummies(trainv1["RFM_R"], drop_first=True)
K = pd.get_dummies(trainv1["LEVEL"], drop_first=True)

In [7]:
trainv1_encoding = pd.concat([trainv1, A, B, C, D, E, F, G, H, I, J, K], axis=1)

In [8]:
trainv1_encoding = trainv1_encoding.drop(["CHARGE_CITY_CD", "CONTACT_CITY_CD", "MARRIAGE_CD", 
                      "CUST_9_SEGMENTS_CD", "APC_1ST_AGE", "REBUY_TIMES_CNT", 
                      "RFM_M_LEVEL", "APC_1ST_YEARDIF", "TERMINATION_RATE", "RFM_R", "LEVEL"], axis=1)

In [9]:
# trainv1_encoding.to_csv("train_cleaned_version1_proc_nontree.csv")

# test dataset

In [10]:
testv1["OCCUPATION_CLASS_CD"] = testv1["OCCUPATION_CLASS_CD"].apply(occ_tran)

In [11]:
testv1["OCCUPATION_CLASS_CD"].value_counts()

1.0    116804
2.0     22388
3.0      5776
4.0      3189
7.0      1180
5.0       346
6.0       317
Name: OCCUPATION_CLASS_CD, dtype: int64

In [12]:
# testv1.to_csv("test_cleaned_version1_proc_tree.csv")

In [13]:
A = pd.get_dummies(testv1["CHARGE_CITY_CD"], drop_first=True)
B = pd.get_dummies(testv1["CONTACT_CITY_CD"], drop_first=True)
C = pd.get_dummies(testv1["MARRIAGE_CD"], drop_first=True)
D = pd.get_dummies(testv1["CUST_9_SEGMENTS_CD"], drop_first=True)
E = pd.get_dummies(testv1["APC_1ST_AGE"], drop_first=True)
F = pd.get_dummies(testv1["REBUY_TIMES_CNT"], drop_first=True)
G = pd.get_dummies(testv1["RFM_M_LEVEL"], drop_first=True)
H = pd.get_dummies(testv1["APC_1ST_YEARDIF"], drop_first=True)
I = pd.get_dummies(testv1["TERMINATION_RATE"], drop_first=True)
J = pd.get_dummies(testv1["RFM_R"], drop_first=True)
K = pd.get_dummies(testv1["LEVEL"], drop_first=True)

In [14]:
testv1_encoding = pd.concat([testv1, A, B, C, D, E, F, G, H, I, J, K], axis=1)

In [15]:
testv1_encoding = testv1_encoding.drop(["CHARGE_CITY_CD", "CONTACT_CITY_CD", "MARRIAGE_CD", 
                      "CUST_9_SEGMENTS_CD", "APC_1ST_AGE", "REBUY_TIMES_CNT", 
                      "RFM_M_LEVEL", "APC_1ST_YEARDIF", "TERMINATION_RATE", "RFM_R", "LEVEL"], axis=1)

In [16]:
# testv1_encoding.to_csv("test_cleaned_version1_proc_nontree.csv")